# Chain Models

Testa modelos usando chaining

## Configuração de Ambiente

In [1]:
# Variáveis de Configuração

# Caminhos de arquivo
raw_data_path = '../data/raw-data/'                 # dados não processados
raw_dataset_path = raw_data_path + 'train_updated.csv'
processed_data_path = '../data/processed-data/'     # dados processados
processed_dataset_path = processed_data_path + 'train.pkl' 
model_path = '../models/trained-models/'

In [2]:
# Imports

# Data manipulation
import pandas as pd
import numpy as np

import pickle as pkl

from joblib import Parallel, delayed

from models import *

# disable warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
warnings.filterwarnings('ignore')

In [3]:
# Variáveis/Constantes Globais
TARGET_COLS = ['target1', 'target2', 'target3', 'target4']

RANDOM_SEED = 42

TEST_SPLIT_DATE = '2021-04-30'

In [4]:
# Carregando o dataframe
df_train = pd.read_pickle(processed_dataset_path)
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2684030 entries, 7 to 2698456
Data columns (total 88 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Dt                        datetime64[ns]
 1   IdPlayer                  int32         
 2   target1                   float16       
 3   target2                   float16       
 4   target3                   float16       
 5   target4                   float16       
 6   target1_shift_1           float16       
 7   target2_shift_1           float16       
 8   target3_shift_1           float16       
 9   target4_shift_1           float16       
 10  target1_shift_2           float16       
 11  target2_shift_2           float16       
 12  target3_shift_2           float16       
 13  target4_shift_2           float16       
 14  target1_shift_3           float16       
 15  target2_shift_3           float16       
 16  target3_shift_3           float16       
 17  target4_

,Dt,IdPlayer,target1,target2,target3,target4,target1_shift_1,target2_shift_1,target3_shift_1,target4_shift_1,...,NuInheritedRunnersScored,NuSaves,NuHolds,NuBlownSaves,NuAssists,NuPutOuts,NuErrors,NuChances,DtHour,DtMinute
7,2018-01-08,112526,0.032349,4.394531,0.039185,17.625000,0.014557,4.714844,0.026337,16.171875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2018-01-09,112526,0.016098,1.791016,0.055420,4.628906,0.032349,4.394531,0.039185,17.625000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2018-01-10,112526,0.075623,1.867188,0.023239,8.507812,0.016098,1.791016,0.055420,4.628906,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,2018-01-11,112526,0.035004,2.285156,0.020096,7.941406,0.075623,1.867188,0.023239,8.507812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2018-01-12,112526,0.018204,0.624023,0.007935,4.925781,0.035004,2.285156,0.020096,7.941406,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Preparação dos Dados

Manter no notebook apenas a preparação pontual dos dados para a aplicação em modelos, toda preparação permanente deve ser adicionada em `/src/data-engineering/data-engineering.ipynb` e então salvo o dataset processado em `/src/data/processed-data/`.

### Funções Auxiliares

Apenas manter aqui funções que serão utilizadas pontualmente no notebook, em caso de funções que serão usadas em outros notebooks, deve-se transferir as mesmas para um arquivo `.py` separado e importar as funções para o notebook.

In [5]:
# Funções Auxiliares

# Imports
from data_preparation import *

# Funções Auxiliares


### Preparação

### Train Test Split

In [6]:
train, test, val = train_test_split(df_train)

In [7]:
train['Dt'] = pd.to_numeric(pd.to_datetime(train['Dt']))
test['Dt']= pd.to_numeric(pd.to_datetime(test['Dt']))
val['Dt'] = pd.to_numeric(pd.to_datetime(val['Dt']))

## Treinando Modelos

In [8]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.multioutput import RegressorChain

from itertools import permutations
chain_list = permutations(TARGET_COLS)

df_results = pd.DataFrame(columns = ['model', 'target1', 'target2', 'target3', 'target4', 'average'])

In [9]:
import matplotlib.pyplot as plt
# train[TARGET_COLS].corr().style.background_gradient(cmap='coolwarm')
train[TARGET_COLS].corr().sum().sort_values(ascending=False)

target2    2.354918
target4    2.319362
target1    2.213762
target3    2.183752
dtype: float64

In [10]:
chain_order = (3, 2, 0, 1) # targets 4, 3, 1, 2

### Baselines

In [11]:
# naive
naive = train[train['Dt']=='2021-04-30'].set_index('IdPlayer')[TARGET_COLS]

# media
media = train[TARGET_COLS].mean()
media_por_jogador = train.groupby('IdPlayer')[TARGET_COLS].mean()

# mediana
mediana = train[TARGET_COLS].median()
mediana_por_jogador = train.groupby('IdPlayer')[TARGET_COLS].median()

In [12]:
# summary = pd.DataFrame()

# for target in TARGET_COLS:
    
#     y_true = test[target]
    
#     mediapj_pred = test['IdPlayer'].map(media_por_jogador[target].to_dict())
#     medianapj_pred = test['IdPlayer'].map(mediana_por_jogador[target].to_dict())
#     naive_pred = test['IdPlayer'].map(naive[target].to_dict())
    
#     mediana_pred = [mediana[target] for i in test.index]
#     media_pred = [media[target] for i in test.index]
    

#     summary.loc['Média',target]  = mean_absolute_error(y_true,media_pred)
#     summary.loc['Média por Jogador',target]  = mean_absolute_error(y_true,mediapj_pred)
#     summary.loc['Mediana',target]  = mean_absolute_error(y_true,mediana_pred)
#     summary.loc['Mediana por Jogador',target]  = mean_absolute_error(y_true,medianapj_pred)
#     summary.loc['Naive',target]  = mean_absolute_error(y_true,naive_pred)
    
# summary['average'] = summary.mean(axis=1)

# summary = summary.reset_index()
# summary = summary.rename(columns = {"index": "model"})
# df_results = df_results.append(summary, ignore_index = True)
# df_results

### Modelos Lineares

#### LASSO

In [13]:
%%time
from sklearn.linear_model import Lasso

# defines the model and create the chain regressor
model = Lasso(alpha=0.1, random_state=RANDOM_SEED)
model_name = 'Lasso | Chain'
regressor = RegressorChain(model, verbose=True, order=chain_order)

# fit the model
regressor.fit(train.drop(columns=TARGET_COLS), train[TARGET_COLS])

# make predictions
y_pred = pd.DataFrame(regressor.predict(test.drop(columns=TARGET_COLS)), columns=TARGET_COLS)

# evaluate the model
mae = evaluate_mae(test[TARGET_COLS], y_pred)

# save the results
df_results = df_results.append({'model': model_name, **mae}, ignore_index=True)
df_results[df_results['model'] == model_name]

[Chain] ................... (1 of 4) Processing order 3, total=  40.6s
[Chain] ................... (2 of 4) Processing order 2, total= 1.1min
[Chain] ................... (3 of 4) Processing order 0, total= 3.7min
[Chain] ................... (4 of 4) Processing order 1, total= 1.9min
CPU times: total: 18min 21s
Wall time: 7min 21s


,model,target1,target2,target3,target4,average
0,Lasso | Chain,0.79089,1.377141,0.714384,0.745432,0.906961


#### Ridge

In [14]:
%%time
from sklearn.linear_model import Ridge

# defines the model and create the multioutput regressor
model = Ridge(alpha=0.1, random_state=RANDOM_SEED)
model_name = 'Ridge | Chain'
regressor = RegressorChain(model, verbose=True, order=chain_order)

# fit the model
regressor.fit(train.drop(columns=TARGET_COLS), train[TARGET_COLS])

# make predictions
y_pred = pd.DataFrame(regressor.predict(test.drop(columns=TARGET_COLS)), columns=TARGET_COLS)

# evaluate the model
mae = evaluate_mae(test[TARGET_COLS], y_pred)

# save the results
df_results = df_results.append({'model': model_name, **mae}, ignore_index=True)
df_results[df_results['model'] == model_name]

[Chain] ................... (1 of 4) Processing order 3, total=   2.7s
[Chain] ................... (2 of 4) Processing order 2, total=   3.0s
[Chain] ................... (3 of 4) Processing order 0, total=   2.5s
[Chain] ................... (4 of 4) Processing order 1, total=   2.6s
CPU times: total: 8.62 s
Wall time: 13.8 s


,model,target1,target2,target3,target4,average
1,Ridge | Chain,0.789278,1.292713,0.737686,0.760436,0.895028


#### ElasticNet

In [15]:
%%time
from sklearn.linear_model import ElasticNet

# defines the model and create the multioutput regressor
model = ElasticNet(alpha=0.1, random_state=RANDOM_SEED)
model_name = 'ElasticNet | Chain'
regressor = RegressorChain(model, verbose=True, order=chain_order)

# fit the model
regressor.fit(train.drop(columns=TARGET_COLS), train[TARGET_COLS])

# make predictions
y_pred = pd.DataFrame(regressor.predict(test.drop(columns=TARGET_COLS)), columns=TARGET_COLS)

# evaluate the model
mae = evaluate_mae(test[TARGET_COLS], y_pred)

# save the results
df_results = df_results.append({'model': model_name, **mae}, ignore_index=True)
df_results[df_results['model'] == model_name]

[Chain] ................... (1 of 4) Processing order 3, total=  46.0s
[Chain] ................... (2 of 4) Processing order 2, total= 8.5min
[Chain] ................... (3 of 4) Processing order 0, total=  58.8s
[Chain] ................... (4 of 4) Processing order 1, total= 5.0min
CPU times: total: 39min 32s
Wall time: 15min 17s


,model,target1,target2,target3,target4,average
2,ElasticNet | Chain,0.789754,1.368599,0.716712,0.747147,0.905553


### Tree Models

In [16]:
%%time
from sklearn.tree import DecisionTreeRegressor

# defines the model and create the multioutput regressor
model = DecisionTreeRegressor(random_state=RANDOM_SEED)
model_name = 'DecisionTreeRegressor | Chain'
regressor = RegressorChain(model, verbose=True, order=chain_order)

# fit the model
regressor.fit(train.drop(columns=TARGET_COLS), train[TARGET_COLS])

# make predictions
y_pred = pd.DataFrame(regressor.predict(test.drop(columns=TARGET_COLS)), columns=TARGET_COLS)

# evaluate the model
mae = evaluate_mae(test[TARGET_COLS], y_pred)

# save the results
df_results = df_results.append({'model': model_name, **mae}, ignore_index=True)
df_results[df_results['model'] == model_name]

[Chain] ................... (1 of 4) Processing order 3, total= 4.6min
[Chain] ................... (2 of 4) Processing order 2, total=22.3min
[Chain] ................... (3 of 4) Processing order 0, total=20.0min
[Chain] ................... (4 of 4) Processing order 1, total= 4.7min
CPU times: total: 23min 20s
Wall time: 51min 41s


,model,target1,target2,target3,target4,average
3,DecisionTreeRegressor | Chain,1.913315,2.397333,1.775872,1.628264,1.928696


#### Gradient Boosting

In [18]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

# defines the model and create the multioutput regressor
model = GradientBoostingRegressor(random_state=RANDOM_SEED)
model_name = 'GradientBoostingRegressor | Chain'
regressor = RegressorChain(model)

# fit the model
regressor.fit(train.drop(columns=TARGET_COLS), train[TARGET_COLS])

# make predictions
y_pred = pd.DataFrame(regressor.predict(test.drop(columns=TARGET_COLS)), columns=TARGET_COLS)

# evaluate the model
mae = evaluate_mae(test[TARGET_COLS], y_pred)

# save the results
df_results = df_results.append({'model': model_name, **mae}, ignore_index=True)
df_results[df_results['model'] == model_name]

## Resultados

In [ ]:
df_results

,model,target1,target2,target3,target4,average
0,Lasso | MultiOutput,0.780746,1.344433,0.703082,0.746220,0.893620
1,Ridge | MultiOutput,0.784643,1.331393,0.729669,0.756929,0.900659
2,ElasticNet | MultiOutput,0.782295,1.342960,0.710303,0.747843,0.895850
3,DecisionTreeRegressor | MultiOutput,2.361119,2.124396,1.964909,1.574683,2.006277
4,GradientBoostingRegressor | MultiOutput,0.731110,1.066651,0.666981,0.726966,0.797927


## Permutando as Chains

In [ ]:
# defines the model and create the multioutput regressor
model = Ridge(alpha=0.1, random_state=RANDOM_SEED)

for chain_order in permutations([0, 1, 2, 3]):

    model_name = 'Ridge | Chain | ' + str(chain_order)

    regressor = RegressorChain(model, verbose=True, order=chain_order)

    # fit the model
    regressor.fit(train.drop(columns=TARGET_COLS), train[TARGET_COLS])

    # make predictions
    y_pred = pd.DataFrame(regressor.predict(test.drop(columns=TARGET_COLS)), columns=TARGET_COLS)

    # evaluate the model
    mae = evaluate_mae(test[TARGET_COLS], y_pred)

    # save the results
    df_results = df_results.append({'model': model_name, **mae}, ignore_index=True)
    df_results[df_results['model'] == model_name]

[Chain] ................... (1 of 4) Processing order 0, total=   2.4s
[Chain] ................... (2 of 4) Processing order 1, total=   2.4s
[Chain] ................... (3 of 4) Processing order 2, total=   2.4s
[Chain] ................... (4 of 4) Processing order 3, total=   2.4s
[Chain] ................... (1 of 4) Processing order 0, total=   2.3s
[Chain] ................... (2 of 4) Processing order 1, total=   2.3s
[Chain] ................... (3 of 4) Processing order 3, total=   2.3s
[Chain] ................... (4 of 4) Processing order 2, total=   2.4s
[Chain] ................... (1 of 4) Processing order 0, total=   2.3s
[Chain] ................... (2 of 4) Processing order 2, total=   2.3s
[Chain] ................... (3 of 4) Processing order 1, total=   2.4s
[Chain] ................... (4 of 4) Processing order 3, total=   2.4s
[Chain] ................... (1 of 4) Processing order 0, total=   2.3s
[Chain] ................... (2 of 4) Processing order 2, total=   2.4s
[Chain

In [ ]:
df_results

,model,target1,target2,target3,target4,average
0,Lasso | Chain,0.790890,1.377141,0.714384,0.745432,0.906961
1,Ridge | Chain,0.789278,1.292713,0.737686,0.760436,0.895028
2,Ridge | Chain,0.789278,1.292713,0.737686,0.760436,0.895028
3,Ridge | Chain,0.789278,1.292713,0.737686,0.760436,0.895028
4,Ridge | Chain,0.789278,1.292713,0.737686,0.760436,0.895028
5,"Ridge | Chain | (0, 1, 2, 3)",0.789278,1.292713,0.737686,0.760436,0.895028
6,"Ridge | Chain | (0, 1, 3, 2)",0.789278,1.292713,0.737686,0.760436,0.895028
7,"Ridge | Chain | (0, 2, 1, 3)",0.789278,1.292713,0.737686,0.760436,0.895028
8,"Ridge | Chain | (0, 2, 3, 1)",0.789278,1.292713,0.737686,0.760436,0.895028
9,"Ridge | Chain | (0, 3, 1, 2)",0.789278,1.292713,0.737686,0.760436,0.895028
